In [ ]:
import numpy as np
import pandas as pd
import random
import uuid
import json
import time 

from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import Runnable
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

from dotenv import load_dotenv



In [ ]:
load_dotenv()  

True

In [27]:
df_retail = pd.read_csv('../data/Retail_Transactions_Dataset.csv')

In [28]:
#df_retail.sample(10)

## Product Set Data Prepration

In [ ]:
ProductSet = set()
Products = pd.DataFrame(columns=['ProductSet','Product'])
for i in range(len(df_retail)):
    a = df_retail.iloc[i]['Product']
    ProductSet.update(a.replace("[",'').replace("]",'').replace("'",'').replace(', ',',').split(','))

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [30]:
Products['Product'] = list(ProductSet)
Products['ProductSet'] = 'Daily Essentials'

In [32]:
Products.shape

(81, 2)

In [33]:
#df_products = pd.DataFrame(data=ProductSet,columns=['Products'])
Products.to_csv('../data/Products.csv',index=False)

In [37]:
list(Products['ProductSet'].unique())

['Daily Essentials']

In [74]:
Products[Products['ProductSet']=='Daily Essentials']['Product']

0                     Ice Cream
1                       Vinegar
2                         Honey
3                   Bath Towels
4     Feminine Hygiene Products
                ...            
76                      Tissues
77                 Power Strips
78                      Sponges
79                         Soap
80                      Shampoo
Name: Product, Length: 81, dtype: object

In [75]:
a = list(Products[Products['ProductSet']=='Daily Essentials']['Product'].unique())
a.sort()
keys = list(range(len(a)))
#dict(zip(keys, a))
dict.fromkeys(['ProductSet'],a)

{'ProductSet': ['Air Freshener',
  'Apple',
  'BBQ Sauce',
  'Baby Wipes',
  'Banana',
  'Bath Towels',
  'Beef',
  'Bread',
  'Broom',
  'Butter',
  'Canned Soup',
  'Carrots',
  'Cereal',
  'Cereal Bars',
  'Cheese',
  'Chicken',
  'Chips',
  'Cleaning Rags',
  'Cleaning Spray',
  'Coffee',
  'Deodorant',
  'Diapers',
  'Dish Soap',
  'Dishware',
  'Dustpan',
  'Eggs',
  'Extension Cords',
  'Feminine Hygiene Products',
  'Garden Hose',
  'Hair Gel',
  'Hand Sanitizer',
  'Honey',
  'Ice Cream',
  'Insect Repellent',
  'Iron',
  'Ironing Board',
  'Jam',
  'Ketchup',
  'Laundry Detergent',
  'Lawn Mower',
  'Light Bulbs',
  'Mayonnaise',
  'Milk',
  'Mop',
  'Mustard',
  'Olive Oil',
  'Onions',
  'Orange',
  'Pancake Mix',
  'Paper Towels',
  'Pasta',
  'Peanut Butter',
  'Pickles',
  'Plant Fertilizer',
  'Potatoes',
  'Power Strips',
  'Razors',
  'Rice',
  'Salmon',
  'Shampoo',
  'Shaving Cream',
  'Shower Gel',
  'Shrimp',
  'Soap',
  'Soda',
  'Spinach',
  'Sponges',
  'Syrup'

In [48]:
a

## User Data Prepration

In [72]:
# Customer_Name,Customer_Type,Gender,Age
df_user = pd.DataFrame()
df_user['Customer_Name'] = df_retail['Customer_Name'].unique()
df_user['guid'] = [str(uuid.uuid4()) for _ in range(len(df_user))]

In [74]:
def returnAge(customerType):
    if customerType == 'Senior Citizen':
        return random.randint(60,80)
    elif customerType == 'Homemaker':
        return random.randint(18,60)
    elif customerType == 'Teenager':
        return random.randint(13,19)
    elif customerType == 'Retiree':
        return random.randint(60,80)
    elif customerType == 'Student':
        return random.randint(5,18)
    elif customerType == 'Professional':
        return random.randint(30,55)
    elif customerType == 'Middle-Aged':
        return random.randint(40,60)
    elif customerType == 'Young Adult':
        return random.randint(18,30)

In [158]:
df_user['Customer_Type'] = np.nan
df_user['Age'] = 0

In [ ]:
for i, row in df_user.iterrows():
    name = row['Customer_Name']
    df_user.at[i,'Customer_Type'] = df_retail[df_retail['Customer_Name'] == name].iloc[0]['Customer_Category']
    df_user.at[i,'Age'] = returnAge(df_user.at[i,'Customer_Type'])
    

C:\Users\Ravi\AppData\Local\Temp\ipykernel_6524\3017680431.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Student' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_user.at[i,'Customer_Type'] = df_retail[df_retail['Customer_Name'] == name].iloc[0]['Customer_Category']


KeyboardInterrupt: 

In [142]:
class genderParser(Runnable):
    def invoke(self,result, tag):
        try:
            a = json.loads(result.content)
            return a['gender']
        except Exception as e:
            print(type(result))
            return np.nan

parser = genderParser()

prompt = PromptTemplate(
    template='What is the usual gender of name {name}? Please answer only M or F in format \"{{\"gender:value\"}}\"',
    input_variables=['name']
)

model_name = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"
llm = HuggingFaceEndpoint(
    repo_id=model_name,
    task='text-generation',
    temperature=0
)

model = ChatHuggingFace(llm=llm)

chain = prompt | model | parser

C:\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
df_user['fname'] = np.nan
df_user['lname'] = np.nan

In [ ]:
for i, row in df_user.iterrows():
    name = row['Customer_Name'].replace('Mrs. ','').replace('Mr. ','').replace('Dr. ','').replace('Ms. ','').split()
    df_user.at[i,'fname'] = name[0]
    df_user.at[i,'lname'] = name[1]


C:\Users\Ravi\AppData\Local\Temp\ipykernel_6524\412664388.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Jeremy' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_user.at[i,'fname'] = name[0]
C:\Users\Ravi\AppData\Local\Temp\ipykernel_6524\412664388.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Irwin' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_user.at[i,'lname'] = name[1]


In [145]:
df_user['Gender'] = np.nan

In [ ]:
for i, row in df_user.iterrows():
    if (row['Gender'] in ['M','F']) == True:
        continue
    try:
        fname, lname = row['Customer_Name'].replace('Mrs. ','').replace('Mr. ','').replace('Dr. ','').replace('Ms. ','').split()
        gender = chain.invoke({'name':fname})
        df_user.loc[df_user['fname'] == fname, 'Gender'] = gender
        if i % 5 == 0:
            time.sleep(5)
    except Exception as e:
        print(i)
        time.sleep(60)

C:\Users\Ravi\AppData\Local\Temp\ipykernel_6524\3891245755.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'F' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_user.loc[df_user['fname'] == fname, 'Gender'] = gender


3


In [ ]:
for i in range(df_user.shape[0]):
    df_user.at[i,'Phone'] = str(9999999999 - i)

In [18]:
df_user.to_csv('../data/UserData.csv',index=False)

## User-Item Data Prepration

In [6]:
# Execute this section if you are executing only this section
df_retail = pd.read_csv('../data/Retail_Transactions_Dataset.csv')
df_products = pd.read_csv('../data/Products.csv')
df_user = pd.read_csv('../data/UserData.csv')

In [7]:
usermap = dict(zip(df_user['Customer_Name'], df_user['guid']))

In [13]:
df_user

,Customer_Name,Customer_Type,Gender,Age,guid
0,Stacey Price,Homemaker,F,25,58526ee0-a7c0-480f-a782-2c0c92b3c8ab
1,Michelle Carlson,Professional,F,47,29206584-b0de-4a9f-a228-2ccd7519d732
2,Lisa Graves,Professional,F,34,1200e410-81c6-4604-aff6-9aa124a692a5
3,Mrs. Patricia May,Homemaker,F,44,a1978ec1-8b08-4f03-b9f2-a23317ee6d5e
4,Susan Mitchell,Young Adult,F,22,03a0b807-9cd6-4fff-94a5-25e12548f300
...,...,...,...,...,...
329740,Mr. James Buchanan,Student,M,6,1880a3a4-184c-4f8e-beef-a646a5688ff3
329741,Gary Mosley,Retiree,M,71,fcb868f3-7e4b-46aa-a276-44ec32c38d75
329742,Gary Salas,Middle-Aged,M,59,1a315d4c-fa8e-43ab-b830-8978630ccc7f
329743,Shaun Wilkerson,Young Adult,M,19,ef9b1c31-c275-4df4-bfd7-562404157004


In [14]:
df_user[df_user['Customer_Name']=='Stacey Price']

,Customer_Name,Customer_Type,Gender,Age,guid
0,Stacey Price,Homemaker,F,25,58526ee0-a7c0-480f-a782-2c0c92b3c8ab


In [10]:
usermap.get('Stacey Price')

'58526ee0-a7c0-480f-a782-2c0c92b3c8ab'

In [15]:
existing_pairs = set()
for i, row in df_retail.iterrows():
    products = row.Product.replace("[",'').replace("]",'').replace("'",'').replace(', ',',').split(',')
    user = usermap.get(row['Customer_Name'])
    existing_pairs.update(zip([user] * len(products), products, [1] * len(products)))
    if i % 10000 == 0:
        print(i,' Columns Completed')

0  Cloums Completed
10000  Cloums Completed
20000  Cloums Completed
30000  Cloums Completed
40000  Cloums Completed
50000  Cloums Completed
60000  Cloums Completed
70000  Cloums Completed
80000  Cloums Completed
90000  Cloums Completed
100000  Cloums Completed
110000  Cloums Completed
120000  Cloums Completed
130000  Cloums Completed
140000  Cloums Completed
150000  Cloums Completed
160000  Cloums Completed
170000  Cloums Completed
180000  Cloums Completed
190000  Cloums Completed
200000  Cloums Completed
210000  Cloums Completed
220000  Cloums Completed
230000  Cloums Completed
240000  Cloums Completed
250000  Cloums Completed
260000  Cloums Completed
270000  Cloums Completed
280000  Cloums Completed
290000  Cloums Completed
300000  Cloums Completed
310000  Cloums Completed
320000  Cloums Completed
330000  Cloums Completed
340000  Cloums Completed
350000  Cloums Completed
360000  Cloums Completed
370000  Cloums Completed
380000  Cloums Completed
390000  Cloums Completed
400000  Cloums

In [16]:
df_useritem = pd.DataFrame(columns=['user','product','rating'])
df_useritem = pd.concat([df_useritem,pd.DataFrame(data=existing_pairs,columns=['user','product','rating'])],ignore_index=True)
print(df_useritem.shape)

(2474923, 3)


In [20]:
CustomerNames = list(df_retail['Customer_Name'].unique())
unrated_pairs = [(usermap.get(user), product,rating) for user in CustomerNames for product in ProductSet for rating in [0]
                 if (user, product) not in existing_pairs]

In [21]:
df_useritem = pd.concat([df_useritem,pd.DataFrame(data=unrated_pairs,columns=['user','product','rating'])],ignore_index=True)
df_useritem.shape

(29183701, 3)

In [24]:
df_useritem.shape

(29183701, 3)

In [22]:
df_useritem.to_csv('../data/UserItemData.csv',index=False)